In [0]:
#/FileStore/tables/CA_youtube_trending_data.csv
import os.path
baseDir = os.path.join('.')
inputPath1 = os.path.join( '/FileStore/tables/Cateogry1Data-2.csv')
fileName1 = os.path.join(baseDir, inputPath1)

CA_youtube_RDD = sc.textFile(fileName1,2)

In [0]:
numPoints = CA_youtube_RDD.count()
print(numPoints)
#Lets take the sample of points
samplePoints = CA_youtube_RDD.take(1) 
print(samplePoints)

15614
['video_id,title,categoryId,view_count,likes,dislikes,comment_count,thumbnail_link,comments_disabled,ratings_disabled,description']


In [0]:
samplePoint2 = CA_youtube_RDD.take(30)
print(samplePoint2)

['video_id,title,categoryId,view_count,likes,dislikes,comment_count,thumbnail_link,comments_disabled,ratings_disabled,description', '5WjcDji3xYc,Honest Trailers | Avatar: The Last Airbender,1,833369,50183,1120,4634,https://i.ytimg.com/vi/5WjcDji3xYc/default.jpg,FALSE,FALSE,"►►Subscribe to ScreenJunkies!► https://fandom.link/SJSubscribeOur Next Fandom 5 Trivia Show is on Avatar: The Last Airbender and will be Friday, August 14th! More Info►► https://fandom.link/AvatarTriviaInfo►►Watch the Honest Trailers Commentary LIVE on Fandom Entertainment► https://youtu.be/Ww9ZHVYLpJM►►100 Reasons Why You Should Love Avatar: The Last Airbender w/ Roth, Joe, & Spencer► https://youtu.be/ml5OSwEqB_UHonest Trailers | Avatar: The Last AirbenderVoice Narration: Jon Bailey aka Epic Voice GuyWritten by: Spencer Gilbert, Joe Starr, Danielle Radford, & Lon HarrisProduced by: Spencer Gilbert & Joe StarrEdited by: Kevin WilliamsenPost-Production Supervisor: Emin BassavandSupervising Producer: Max DionneAssocia

In [0]:
def extractFields(CA_youtubeRDD_Record):
    #Create a key-value pair consisting of : 
    #Video Id, title, categoryID, view_count, likes, dislikes, comment_count
    fieldList = CA_youtubeRDD_Record.split(',') #Delimited by , 
    return(fieldList[0], (fieldList[1], fieldList[5], fieldList[8], fieldList[9], fieldList[10]))

CA_youtube_PairRDD = CA_youtube_RDD.map(extractFields)
print(CA_youtube_PairRDD.take(2))

[('video_id', ('title', 'dislikes', 'comments_disabled', 'ratings_disabled', 'description')), ('5WjcDji3xYc', ('Honest Trailers | Avatar: The Last Airbender', '1120', 'FALSE', 'FALSE', '"►►Subscribe to ScreenJunkies!► https://fandom.link/SJSubscribeOur Next Fandom 5 Trivia Show is on Avatar: The Last Airbender and will be Friday'))]


In [0]:
def filterData (CA_youtubeRDD_Record):
    fieldList = CA_youtubeRDD_Record.split(',')
    if len(fieldList)<0 :
        return False 
    else :
        return True 
    
filteredRDD = CA_youtube_RDD.filter(filterData)
#print(filteredRDD.take(5))


In [0]:
print(CA_youtube_PairRDD.take(5))

[('video_id', ('title', 'dislikes', 'comments_disabled', 'ratings_disabled', 'description')), ('5WjcDji3xYc', ('Honest Trailers | Avatar: The Last Airbender', '1120', 'FALSE', 'FALSE', '"►►Subscribe to ScreenJunkies!► https://fandom.link/SJSubscribeOur Next Fandom 5 Trivia Show is on Avatar: The Last Airbender and will be Friday')), ('z5l8ovbw_6M', ("Don't be a Tourist", '876', 'FALSE', 'FALSE', 'This one is for all you full time travelersEMAIL FOR INQUIRIES:Tabbesk@gmail.com TIKTOK:https://www.tiktok.com/@tabtabbes?lang=enMERCH:https://crowdmade.com/collections/tabbesDISCORD: https://discord.gg/TabbesINSTAGRAM: https://www.instagram.com/tabtabbes/FACEBOOK: https://www.facebook.com/tab.tabbes/TWITTER:https://twitter.com/TabTabbesWEBTOON: https://www.webtoons.com/en/challenge/paper-crown/list?title_no=265219')), ('yVdH3QacEXc', ('Selena Gomez - This is the Year (Official Premiere Event)', '2377', 'FALSE', 'FALSE', '"Get your tickets here: https://thisistheyear.film Join Selena and David

In [0]:
from pyspark.mllib.regression import LabeledPoint
import numpy as np

In [0]:
%sh
/databricks/python3/bin/pip install spacy 
/databricks/python3/bin/python3 -m spacy download en_core_web_sm

You should consider upgrading via the '/databricks/python3/bin/python -m pip install --upgrade pip' command.
  Using cached https://github.com/explosion/spacy-models/releases/download/en_core_web_sm-3.7.1/en_core_web_sm-3.7.1-py3-none-any.whl (12.8 MB)
You should consider upgrading via the '/databricks/python3/bin/python3 -m pip install --upgrade pip' command.
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [0]:
%sh
/local_disk0/.ephemeral_nfs/envs/pythonEnv-87fd4857-b3b5-49b0-b2d3-1f35732d5d77/bin/python -m pip install --upgrade pip

/bin/bash: /local_disk0/.ephemeral_nfs/envs/pythonEnv-87fd4857-b3b5-49b0-b2d3-1f35732d5d77/bin/python: No such file or directory


In [0]:
%sh
pip install "https://github.com/explosion/spacy-models/releases/download/en_core_web_md-2.2.5/en_core_web_md-2.2.5.tar.gz"

You should consider upgrading via the '/local_disk0/.ephemeral_nfs/envs/pythonEnv-c74583d8-9202-4741-857e-7bf962307ee1/bin/python -m pip install --upgrade pip' command.


In [0]:
dbutils.library.restartPython()

In [0]:
import spacy


df = spark.sql("SELECT title FROM cateogry1data_2_csv")
titles = df.filter(df["title"].isNotNull() & (df["title"] != "")).rdd.map(lambda row: row['title']).collect()
nlp = spacy.load('en_core_web_sm')
title_vectors = []
for title in titles:
    doc = nlp(title)
    #print(doc.vector_norm)
    title_vector = doc.vector_norm
    title_vectors.append(title_vector)


In [0]:
import numpy as np

df1 = spark.sql("SELECT view_count FROM cateogry1data_2_csv")
views = df1.filter(df1["view_count"].isNotNull() & (df1["view_count"] != "")).rdd.map(lambda row: row['view_count']).collect()
views_vector = []
for view in views:
    views_vector.append(view)

title_vectors_trimmed = title_vectors[:len(views_vector)]

# Convert views_vector to a numpy array
views_vector_np = np.array(views_vector)

# Convert title_vectors_trimmed to a numpy array
title_vectors_trimmed_np = np.array(title_vectors_trimmed)
title_vectors_trimmed_np = title_vectors_trimmed_np.reshape(-1, 1)  # Reshaping to a column vector

# Concatenate vertically (along axis=0)
X_combined = np.concatenate((title_vectors_trimmed_np, views_vector_np[:, None]), axis=1)

print(X_combined)

[['4.687736088222961' '833369']
 ['4.509727210116285' '1061892']
 ['3.298016926826576' '1523818']
 ...
 ['4.317405521668583' '7399943']
 ['5.22774597691059' '204157']
 ['4.608736429428359' '2155337']]


In [0]:
y = np.array(views_vector)
y_trimmed = y[:len(views_vector)]
print("Feature Mat X : ")
print(X_combined)
print("Target Y : ")
print(y)

print(X_combined.shape)
print(len(y_trimmed))



Feature Mat X : 
[['4.687736088222961' '833369']
 ['4.509727210116285' '1061892']
 ['3.298016926826576' '1523818']
 ...
 ['4.317405521668583' '7399943']
 ['5.22774597691059' '204157']
 ['4.608736429428359' '2155337']]
Target Y : 
['833369' '1061892' '1523818' ... '7399943' '204157' '2155337']
(8949, 2)
8949
[' Alicia Vikander and Joel Edgerton Like The Green Knight on Facebook: http://bit.ly/TheGreenKnight_facebookFollow The Green Knight on Twitter: http://bit.ly/TheGreenKnight_twitterFollow the Green Knight on Instagram: http://bit.ly/Thegreenknight_instagram'
 ' Arian Moayed' ' Brendan Gleeson' ... '999648' '999677' '999977']
[' Alicia Vikander and Joel Edgerton Like The Green Knight on Facebook: http://bit.ly/TheGreenKnight_facebookFollow The Green Knight on Twitter: http://bit.ly/TheGreenKnight_twitterFollow the Green Knight on Instagram: http://bit.ly/Thegreenknight_instagram'
 ' Arian Moayed' ' Brendan Gleeson' ... '999648' '999677' '999977']


In [0]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

X_combined = np.array(X_combined) if isinstance(X_combined, float) else X_combined
y_trimmed = np.array(y_trimmed) if isinstance(y_trimmed, float) else y_trimmed

# Split the dataset into training and testing sets (e.g., 80-20 split)
X_train, X_test, y_train, y_test = train_test_split(X_combined, y_trimmed, test_size=0.2, random_state=42)

# Initialize and train the model
model = LinearRegression()
model.fit(X_train, y_train)

# Evaluate the model's performance on the test set
predicted = model.predict(X_test)
mse = mean_squared_error(y_test, predicted)
print("Mean Squared Error:", mse)

/databricks/python/lib/python3.9/site-packages/sklearn/utils/validation.py:964: FutureWarning: Arrays of bytes/strings is being converted to decimal numbers if dtype='numeric'. This behavior is deprecated in 0.24 and will be removed in 1.1 (renaming of 0.26). Please convert your data to numeric values explicitly instead.
  X = check_array(


---------------------------------------------------------------------------
ValueError                                Traceback (most recent call last)
File /databricks/python/lib/python3.9/site-packages/sklearn/utils/validation.py:787, in check_array(array, accept_sparse, accept_large_sparse, dtype, order, copy, force_all_finite, ensure_2d, allow_nd, ensure_min_samples, ensure_min_features, estimator)
    786 try:
--> 787     array = array.astype(np.float64)
    788 except ValueError as e:

ValueError: could not convert string to float: ' Tim Meadows'

The above exception was the direct cause of the following exception:

ValueError                                Traceback (most recent call last)
File <command-1071113111622439>:14
     12 # Initialize and train the model
     13 model = LinearRegression()
---> 14 model.fit(X_train, y_train)
     16 # Evaluate the model's performance on the test set
     17 predicted = model.predict(X_test)

File /databricks/python_shell/dbruntime/MLWor